In [ ]:
from xgboost import plot_importance
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import sklearn.metrics
from sklearn.model_selection import GridSearchCV
import pymysql
from sqlalchemy import create_engine
%matplotlib inline

In [ ]:
#172.30.1.211 서버내 test DB 생성 후 연결
engine = create_engine("mysql+pymysql://{인턴과제}/smart_doctor?charset=utf8",convert_unicode=True)
conn = engine.connect()
data = pd.read_sql_table('ODS_QUALITY',conn)

In [ ]:
# 합 불 개수
data.iloc[:,-1].value_counts()
original_data = data.iloc[:,-1]

In [ ]:
# 비율 확인
data.iloc[:, -1].value_counts()/len(data)*100

0    85.042735
1    11.794872
2     2.051282
3     0.769231
4     0.341880
Name: DEFECT_QTY, dtype: float64

In [ ]:
# 합불 판별에 영향을 주는 4가지 컬럼 선택 - 판다스 getdummy함수 이용(범주형 변수 ->연속형 가변수)
testsel = data.loc[:, ['EQUIPMENT_NM', 'ACT_QTY','DEFECT_KIND']]
testdum = pd.get_dummies(testsel)

In [ ]:
X = testdum.loc[:]
y = data.iloc[:, -1]

In [ ]:
#테스트, 학습 데이터 분할 -> 25프로 = 테스트 데이터셋
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=10)

In [ ]:
xgb = XGBClassifier()

xgb_param_grid ={
    'n_estimators' : [100,200,400,600],
    'learning_rate' : [0.01,0.05,0.1,0.15,0.2],
    'max_depth' : [4,6,8,10,12]
}
xgb_grid = GridSearchCV(xgb, param_grid=xgb_param_grid, scoring= 'accuracy', n_jobs=-1, verbose=1)
xgb_grid.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\Atech\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done 438 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.3min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_es...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,

In [ ]:
# 파라미터 평가
print("최고 평균 정확도 : {0:4f}".format(xgb_grid.best_score_))
print("최고의 파라미터 :", xgb_grid.best_params_)

최고 평균 정확도 : 0.882552
최고의 파라미터 : {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100}


In [ ]:
#최적의 값 확인 교차검증
cv_result_df = pd.DataFrame(xgb_grid.cv_results_)
cv_result_df.sort_values(by=['rank_test_score'],inplace=True)

In [ ]:
xgb = XGBClassifier(n_estimators=100, learning_rate = 0.01, max_depth=4)
xgb.fit(X_train,y_train)
xgb_pred = xgb.predict(X_test)

In [ ]:
#xgboost 점수 평가
print('정확도 : {:.2f}'.format(xgb.score(X_test,y_test)))

정확도 : 0.90


In [ ]:
#스태킹 앙상블을 위한 부스팅알고리즘 선언
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
ada_clf = AdaBoostClassifier(n_estimators=100)

In [ ]:
#마지막 분석은 회귀 분석으로 진행
lr_final = LogisticRegression(C=10)

In [ ]:
rf_clf.fit(X_train,y_train)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=None)

In [ ]:
from sklearn.metrics import accuracy_score
rf_pred = rf_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print('rf 정확도: {0:.4f}'.format(accuracy_score(y_test,rf_pred)))
print('ada 정확도: {0:.4f}'.format(accuracy_score(y_test,ada_pred)))

rf 정확도: 0.8840
ada 정확도: 0.7304


In [ ]:
# from sklearn.metrics import accuracy_score, precision_score , recall_score , confusion_matrix,f1_score, roc_auc_score
# 정확도 측정 함수
# def get_clf_eval(y_test , pred=None, pred_proba=None):
#     confusion = confusion_matrix( y_test, pred)
#     accuracy = accuracy_score(y_test , pred)
#     precision = precision_score(y_test , pred)
#     recall = recall_score(y_test , pred)
#     f1 = f1_score(y_test, pred)
#     print('오차 행렬')
#     print(confusion)
#     #roc-auc 추가
#     roc_auc=roc_auc_score(y_test, pred_proba)
#     print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, f1:{3:.4f}, AUC:{4:.4f}'.format(accuracy, precision ,recall,f1,roc_auc))

In [ ]:
pred = np.array([rf_pred,ada_pred,xgb_pred])
# print(pred.shape)
pred = np.transpose(pred)
# print(pred.shape)

In [ ]:
lr_final.fit(pred,y_test)
final = lr_final.predict(pred)
print('스태킹 앙상블 결과 : {0:.4f}'.format(accuracy_score(y_test,final)))

스태킹 앙상블 결과 : 0.9010


In [ ]:
# DB 인서트
final_df = pd.DataFrame(final)
final_df.to_sql(name='ODS_PREDICT_QTY',if_exists="append",con=engine,index=False)